In [1]:
#from FinalID3 import *
import seaborn as sns
import random
import sys

In [2]:
sys.setrecursionlimit(3000)

In [3]:
RFTreeNumber =100
RFDecisionTrees = []
RFPredictedList = []
targetValue = []

In [4]:
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Abalone Feature Correlation')
    labels=[list(df)]
    ax1.set_xticklabels(labels,fontsize=6)
    ax1.set_yticklabels(labels,fontsize=6)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[.75,.8,.85,.90,.95,1])
    plt.show()



In [5]:
def most_common(lst):
    return max(set(lst), key= lst.count)

In [6]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from matplotlib import pylab as plt
################################################################################################################
############################ Global Initialization #############################################################
targetAttribute = 'Poisonous/Edible'
targetValue = []
predictedList= []
minimalGainThreshold = 1e-6
defaultTargetValue = 'p'
rawDataSet = 'MushroomDataSet_Before_PreProcessing.xlsx'
processedDataSet = 'MushroomDataSet_After_PreProcessing.xlsx'
columnToProcess = 'stalk-root'
res = {}
################################################################################################################
################################### Finding the set of Target Value ############################################
def initializeList(data):
    groupedData = data.groupby([targetAttribute]).groups.keys()
    return groupedData
################################################################################################################
################################## Creating the Tree Recursively ###############################################
def createTree(df):
    x = df.drop(labels=targetAttribute, axis=1)
    y = df[targetAttribute] 
    ##Returning the original Set if no further split can be made######## 
    if len(y) == 0:
        return y

    if trueSet(y):
        return set(y).pop()
    
    ##Finding the attribute which gives maximal information#############
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
##If the information gain is less than the threshold set we classify as 'Poisonous' instead of saying it edible##
    if((maxGainAttr== None) and (maxInfoGain == None)):
        return defaultTargetValue
##################################################################################################################
##### If there is an information gain we recursively do splits ##################################################
    sets = df.groupby(maxGainAttr)
    for k, v in sets:
        res["%s = %s" % (maxGainAttr, k)] = createTree(v)
    return res

In [7]:
def calcInforGainForEachAtrribute(df):
    # return maximum info gain
    maxInfoGain =0;
    infoGainDict = {}
    attributes = list(df)
    totalEntropy = entropy(df)

    for att in attributes:
        #don't calculate gain for target attribute
        if (att ==targetAttribute ):
            continue
        grouped_data = df.groupby([att], as_index=False)
        totalRows = len(df)
        subEntropy = 0
        for key, value in grouped_data:
            eachGroupRows = len(value)
            S = entropy(value)
            #calculate |Sv/S|
            valEntrpy = eachGroupRows / totalRows
            #calculate |Sv/S|*Entropy(Sv)
            subEntropy = subEntropy + valEntrpy * S
        individualEntrpyGain = totalEntropy - subEntropy
        infoGainDict[att] = individualEntrpyGain
        if (individualEntrpyGain > maxInfoGain):
            maxInfoGain = individualEntrpyGain
            maxGainAttr = att;
    # If there's no gain at all, nothing has to be done, just return the original set
    if (maxInfoGain < minimalGainThreshold):
        return None, None
    else:
        return maxGainAttr,maxInfoGain
##################################################################################################################
######################### Evaluating if a Pure Set is reached ?################################################### 
def trueSet(s):
    return len(set(s)) == 1
###################################################################################################################
########################### Entropy Calculator ######################################################################
def entropy(df):
    #target unique value calculation in form of dictionary 
    targetDict = pd.value_counts(df[targetAttribute]).to_dict()
    dfLength =len(df) 
    totalEntropy = 0
    for key, val in targetDict.items():
        multipleVal = val/dfLength
        totalEntropy += multipleVal * math.log2(multipleVal)
    return 0-totalEntropy
##################################################################################################################
########################## Evaluating the Test Data by Passing the Learned tree and Test Data as Parameters #####

##################################################################################################################
########################### Printing the Confusion Matrix #######################################################
def printMatrix(y_test,predictedList,labels):
    cm = confusion_matrix(y_test, pd.Series(predictedList), labels)
    print(cm)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Confusion matrix of the classifier')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [8]:
def predictTarget(targetDict,test):
    for k,v in targetDict.items():
        keyList = k.split('=')
        col = keyList[0].strip()
        val = keyList[1].strip()
        if(test[col].iloc[0] == val):
            if k in targetDict:
                temp = targetDict.get(k)
                if(type(temp) == dict):
                    return predictTarget(v,test)
                else:
                    return v

In [9]:
def predictTarget1(targetDict,test):
    t1 =[]
    t2 =[]
    for k,v in targetDict.items():
        keyList = k.split('=')
        col = keyList[0].strip()
        val = keyList[1].strip()
        t1.append(val)
        t2.append(v)
    
    searchVal = test[col].iloc[0]
    if(searchVal in t1 ):
        index = t1.index(searchVal)
        finalVal = t2[index]
        
        if(type(finalVal) == dict):
            return predictTarget1(finalVal,test)
        else:
            return finalVal    
    else:
        return defaultTargetValue

In [10]:
def predictTarget2(targetDict,test):
    t1 =[]
    t2 =[]
    for k,v in targetDict.items():
        keyList = k.split('=')
        col = keyList[0].strip()
        val = keyList[1].strip()
        t1.append(val)
        t2.append(v)
    
    searchVal = test[col].iloc[0]
    if(searchVal in t1 ):
        index = t1.index(searchVal)
        finalVal = t2[index]
        
        return finalVal    
    else:
        return defaultTargetValue

In [31]:
def depth(d, level=1):
    if not isinstance(d, dict) or not d:
        return level
    return max(depth(d[k], level + 1) for k in d)

In [33]:
def main():
    datasetAfterProcessing = pd.read_excel(processedDataSet)
    X = datasetAfterProcessing.drop(labels=targetAttribute, axis=1)
    y = datasetAfterProcessing[targetAttribute]
    print(list(datasetAfterProcessing))
    print(datasetAfterProcessing.shape)
    #results = datasetAfterProcessing.corr()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)
    X_train[targetAttribute] = y_train
    targetValue = initializeList(datasetAfterProcessing)
    dfLength = len(X)
    
    #Mashroom has total length of training data = 6499
    
    #devide total training data by RFTreeNumber
    myset = set()
    #temp = []
    RFDecisionTrees =[]
    '''
    X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_train, y_train, test_size=0.2, random_state=40)
    X_sub_train[targetAttribute] = y_sub_train
        
    #for each chunk create decision tree and store it in RFDecisionTrees
    targetDict = createTree(X_sub_train)
    
    print("depth :",depth(targetDict))
    '''
    
    for i in range(40,70):
        #creating 
        #random_state = random.randint(0,100)
        X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_train, y_train, test_size=0.2, random_state=i)
        X_sub_train[targetAttribute] = y_sub_train
        
        #for each chunk create decision tree and store it in RFDecisionTrees
        targetDict = createTree(X_sub_train)
        #temp.append(str(targetDict))
        RFDecisionTrees.append(targetDict)
    
    #myset = set(temp)
    #print("len :",len(myset))
    
    RFPredictedList =[]
    #print("RFDecisionTrees :",len(RFDecisionTrees))
    for i in range(0,len(X_test)):
        tempPredictionList =[]
        curr = X_test.iloc[[i]]
        for tree in RFDecisionTrees:
            
            myTree = tree
            ansFlag = True
            while(ansFlag):
                tempPredictedval = predictTarget2(myTree,curr)
                if(type(tempPredictedval) == dict):
                    myTree = tempPredictedval
                else:
                    ansFlag = False
            
            #predictedVal = predictTarget(tree,X_test.iloc[[i]])
            tempPredictionList.append(tempPredictedval)
        
        print("length :",len(tempPredictionList),"LIST :",set(tempPredictionList)," most_common(tempPredictionList):",most_common(tempPredictionList))
        RFPredictedList.append(set(most_common(tempPredictionList)).pop())
    
    print("y_test: ",len(y_test))
    print("RFPredictedList: ",len(RFPredictedList))
    printMatrix(y_test,RFPredictedList,list(targetValue))
    
   
if __name__ == "__main__": main()

['Poisonous/Edible', 'cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
(8124, 23)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


RecursionError: maximum recursion depth exceeded